In [1]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")
from millify import millify

In [2]:
from pathlib import Path
p = Path().cwd()
print(f"original File path: {p}")
stringpath= str(p)[0:str(p).rfind('\\')] + '\\data'
print(f"Updated File Path to pick from Data Library: {stringpath}")
ngsimfile=stringpath + '/' + 'Cleaned_NGSIM_Data_Part_1.csv'

ngsim = pd.read_csv(ngsimfile,low_memory=False)
print(f"{ngsim.duplicated().sum()} duplicate values have been removed")
ngsim.drop_duplicates(inplace=True)

original File path: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\scripts
Updated File Path to pick from Data Library: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\data
0 duplicate values have been removed


In [3]:
ngsim['Velocity Difference_Following-Preceding'] = np.NaN
ngsim['Acceleration Difference_Following-Preceding'] = np.NaN

filtered_ngsim_U= ngsim[((ngsim['Location'] == 'us-101'))  ]

filtered_ngsim_U= ngsim[((ngsim['Location'] == 'us-101')) & ((ngsim['Vehicle_ID'] <= 100 )  ) & ((ngsim['Preceding'] <= 100 )  )]
filtered_ngsim_I= ngsim[(ngsim['Location'] == 'i-80') ]


In [4]:
filtered_ngsim_U.shape

(154804, 25)

In [5]:
filtered_ngsim_U.set_index(['Vehicle_ID', 'Preceding','Relative_Time'], append=True,inplace=True)
filtered_ngsim_U=filtered_ngsim_U.sort_index()
print(filtered_ngsim_U.index.is_monotonic_increasing)

True


In [6]:
filtered_ngsim_U.index.rename(['ix', 'Vehicle_ID', 'Preceding', 'Relative_Time'], inplace=True)
filtered_ngsim_U.index

MultiIndex([(     11,  5,  0, 1117684841001),
            (     44, 52, 48, 1117685817401),
            (     51, 73, 67, 1117684928901),
            (     91,  6,  2, 1117685787801),
            (    112, 41, 35, 1117684872901),
            (    122, 77, 72, 1117685822701),
            (    133, 14, 10, 1117684889001),
            (    139, 73, 63, 1117684012201),
            (    176, 26, 22, 1117683969901),
            (    189, 62, 58, 1117684893801),
            ...
            (4098616, 91, 86, 1117685859701),
            (4098632, 83, 67, 1117684011101),
            (4098637, 11,  3, 1117685766501),
            (4098661, 74, 73, 1117685787201),
            (4098664, 85, 80, 1117684887001),
            (4098686, 14, 11, 1117685780801),
            (4098797, 83, 76, 1117683997501),
            (4098811, 83, 90, 1117684899701),
            (4098864, 39, 32, 1117684866701),
            (4098873, 75, 68, 1117685841201)],
           names=['ix', 'Vehicle_ID', 'Preceding', 'Relative_Ti

In [13]:
velocity_diff=[]
acceleration_diff=[]
previous_vehicle_acceleration=[]
previous_vehicle_velocity=[]
counter=0
for i in filtered_ngsim_U.index.get_level_values(0):
    counter=1
    if counter%10000==0:
        print(f"row count{millify(counter, precision=2, drop_nulls=False)} processed")

    previous_Vehicle_ID=filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("ix") == i )].index.get_level_values("Preceding").values[0]
    current_time=filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("ix") == i )].index.get_level_values("Relative_Time")[0]
    #print(f"i:{i}")
    #print(f"previous_Vehicle_ID: {previous_Vehicle_ID}")
    #print(f"current_time: {current_time}")
    if previous_Vehicle_ID > 0:
        preceding_corresponding_ix=filtered_ngsim_U.xs((current_time,previous_Vehicle_ID),level=("Relative_Time",'Vehicle_ID')).index.get_level_values('ix').values[0] 
        previous_vehicle_v=filtered_ngsim_U['v_Vel'][(filtered_ngsim_U.index.get_level_values('ix') == preceding_corresponding_ix ) ].values
        velocity = filtered_ngsim_U['v_Vel'][(filtered_ngsim_U.index.get_level_values('ix') == i ) ].values - previous_vehicle_v
        previous_vehicle_acc=filtered_ngsim_U['v_Acc'][(filtered_ngsim_U.index.get_level_values('ix') == preceding_corresponding_ix ) ].values
        acceleration = filtered_ngsim_U['v_Acc'][(filtered_ngsim_U.index.get_level_values('ix') == i ) ].values - previous_vehicle_acc
    else:
        velocity = filtered_ngsim_U['v_Vel'][(filtered_ngsim_U.index.get_level_values('ix') == i ) ].values 
        acceleration = filtered_ngsim_U['v_Acc'][(filtered_ngsim_U.index.get_level_values('ix') == i ) ].values 
        previous_vehicle_v=0
        previous_vehicle_acc=0
    #print(preceding_corresponding_ix)
    #print(f"velocity previous :{ filtered_ngsim_U['v_Vel'][(filtered_ngsim_U.index.get_level_values('id') == preceding_corresponding_id ) ].values}")
    #print(f"velocity current : { filtered_ngsim_U['v_Vel'][(filtered_ngsim_U.index.get_level_values('id') == i ) ].values}")
#    if i >= 1000:
#        break
    previous_vehicle_velocity.append(previous_vehicle_v)
    previous_vehicle_acceleration.append(previous_vehicle_acc)
    velocity_diff.append(velocity[0])
    acceleration_diff.append(acceleration[0])

In [14]:
filtered_ngsim_U['Velocity Difference_Following-Preceding'] = velocity_diff
filtered_ngsim_U['Acceleration Difference_Following-Preceding'] = acceleration_diff
filtered_ngsim_U['previous_Velocity'] = previous_vehicle_velocity
filtered_ngsim_U['previous_Acceleration'] = previous_vehicle_acceleration

In [ ]:
'''
filtered_ngsim_U['velocity_diff'] = filtered_ngsim_U.apply(lambda x: f(x))

velocity_diff1=[]
acceleration_diff1=[]
counter=0
f= lambda x,y
def f(x) -> pd.Series:
    previous_Vehicle_ID=filtered_ngsim_U[(x.index.get_level_values("ix") == i )].index.get_level_values("Preceding").values[0]
    current_time=x[(x.index.get_level_values("ix") == i )].index.get_level_values("Relative_Time")[0]
    #print(f"i:{i}")
    #print(f"previous_Vehicle_ID: {previous_Vehicle_ID}")
    #print(f"current_time: {current_time}")
    if previous_Vehicle_ID > 0:
        preceding_corresponding_ix=x.xs((current_time,previous_Vehicle_ID),level=("Relative_Time",'Vehicle_ID')).index.get_level_values('ix').values[0] 
        velocity = x['v_Vel'][(x.index.get_level_values('ix') == i ) ].values - x['v_Vel'][(x.index.get_level_values('ix') == preceding_corresponding_ix ) ].values
        acceleration = x['v_Acc'][(filtered_ngsim_U.index.get_level_values('ix') == i ) ].values - x['v_Acc'][(x.index.get_level_values('ix') == preceding_corresponding_ix ) ].values
    else:
        velocity = filtered_ngsim_U['v_Vel'][(filtered_ngsim_U.index.get_level_values('ix') == i ) ].values 
        acceleration = filtered_ngsim_U['v_Acc'][(filtered_ngsim_U.index.get_level_values('ix') == i ) ].values 
    #print(preceding_corresponding_ix)
    #print(f"velocity previous :{ filtered_ngsim_U['v_Vel'][(filtered_ngsim_U.index.get_level_values('id') == preceding_corresponding_id ) ].values}")
    #print(f"velocity current : { filtered_ngsim_U['v_Vel'][(filtered_ngsim_U.index.get_level_values('id') == i ) ].values}")
#    if i >= 1000:
#        break    
    velocity_diff.append(velocity[0])
    acceleration_diff.append(acceleration[0])
    '''

In [15]:
filtered_ngsim_U.head()
#filtered_ngsim_U.xs((1117685909701,242),level=("Relative_Time",'Vehicle_ID')).index.get_level_values('id').values[0] 

,,,,Frame_ID,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Class,v_Vel,v_Acc,...,Preceding_Vehicle_Class,Rear_to_Front_Space_Headway,Front_To_Rear_Time_Headway,part1_for_loop_output_mean_preceding_vehicle_length,v_Class_Name,Vehicle_combination,Velocity Difference_Following-Preceding,Acceleration Difference_Following-Preceding,previous_Velocity,previous_Acceleration
ix,Vehicle_ID,Preceding,Relative_Time,,,,,,,,,,,,,,,,,,,,,
11,5,0,1117684841001,181,1118847860100,4.948,149.178,6451195.587,1873298.132,3.9624,2,0.000000,0.000000,...,NaN,NaN,NaN,NaN,Car,-Car,0.000000,0.000000,0,0
44,52,48,1117685817401,795,1118848836500,42.059,1471.840,6452155.548,1872378.779,4.5720,2,9.750552,-0.045720,...,Car,NaN,NaN,NaN,Car,Car-Car,0.606552,-0.045720,[9.144],[0.0]
51,73,67,1117684928901,1060,1118847948000,18.751,1853.192,6452453.921,1872148.756,4.8768,2,19.299936,-3.413760,...,Car,NaN,NaN,NaN,Car,Car-Car,1.341120,-2.298192,[17.958816000000002],[-1.115568]
91,6,2,1117685787801,499,1118848806900,30.783,1083.438,6451870.511,1872642.596,4.7244,2,10.658856,-0.054864,...,Car,NaN,NaN,NaN,Car,Car-Car,-0.009144,-0.054864,[10.668],[0.0]
112,41,35,1117684872901,500,1118847892000,5.515,217.059,6451241.586,1873248.046,4.4196,2,7.162800,0.000000,...,Car,NaN,NaN,NaN,Car,Car-Car,-3.093720,0.000000,[10.25652],[0.0]


In [ ]:
'''
i=546
previous_Vehicle_ID=filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == i )].index.get_level_values("Preceding").values[0]
current_time=filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == i )].index.get_level_values("Relative_Time")[0]
print(f"i:{i}")
print(f"previous_Vehicle_ID: {previous_Vehicle_ID}")
print(f"current_time: {current_time}")
if previous_Vehicle_ID > 0:
    preceding_corresponding_ix=filtered_ngsim_U.xs((current_time,previous_Vehicle_ID),level=("Relative_Time",'Vehicle_ID')).index.get_level_values('id').values[0] 
    velocity = filtered_ngsim_U['v_Vel'][(filtered_ngsim_U.index.get_level_values('id') == i ) ].values - filtered_ngsim_U['v_Vel'][(filtered_ngsim_U.index.get_level_values('id') == preceding_corresponding_ix ) ].values
else:
    velocity = filtered_ngsim_U['v_Vel'][(filtered_ngsim_U.index.get_level_values('id') == i ) ].values - 0

'''


'\ni=546\nprevious_Vehicle_ID=filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == i )].index.get_level_values("Preceding").values[0]\ncurrent_time=filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == i )].index.get_level_values("Relative_Time")[0]\nprint(f"i:{i}")\nprint(f"previous_Vehicle_ID: {previous_Vehicle_ID}")\nprint(f"current_time: {current_time}")\nif previous_Vehicle_ID > 0:\n    preceding_corresponding_ix=filtered_ngsim_U.xs((current_time,previous_Vehicle_ID),level=("Relative_Time",\'Vehicle_ID\')).index.get_level_values(\'id\').values[0] \n    velocity = filtered_ngsim_U[\'v_Vel\'][(filtered_ngsim_U.index.get_level_values(\'id\') == i ) ].values - filtered_ngsim_U[\'v_Vel\'][(filtered_ngsim_U.index.get_level_values(\'id\') == preceding_corresponding_ix ) ].values\nelse:\n    velocity = filtered_ngsim_U[\'v_Vel\'][(filtered_ngsim_U.index.get_level_values(\'id\') == i ) ].values - 0\n\n'

In [ ]:
'''
filtered_ngsim_U.xs((filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == 546 )].index.get_level_values("Relative_Time")[0],filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == 546 )].index.get_level_values("Preceding").values[0]),level=("Relative_Time",'Vehicle_ID')).index.get_level_values('id').values[0] 
ngsim[(ngsim['Vehicle_ID'] == 242)& (ngsim['Relative_Time'] == 1117685909701) ]
filtered_ngsim_U.xs((filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == 0 )].index.get_level_values("Relative_Time")[0],filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == 0 )].index.get_level_values("Preceding").values[0]),level=("Relative_Time",'Vehicle_ID')).index.get_level_values('id').values[0] 
filtered_ngsim_U.index.get_level_values("Preceding")
ngsim[(ngsim['Vehicle_ID'] == 242)|(ngsim['Preceding'] == 242) ].to_csv('ng', index = True)
filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("Vehicle_ID") == 242 )].sort_values("Relative_Time").to_csv('test', index = True)
                            #1117685909701
'''

'\nfiltered_ngsim_U.xs((filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == 546 )].index.get_level_values("Relative_Time")[0],filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == 546 )].index.get_level_values("Preceding").values[0]),level=("Relative_Time",\'Vehicle_ID\')).index.get_level_values(\'id\').values[0] \nngsim[(ngsim[\'Vehicle_ID\'] == 242)& (ngsim[\'Relative_Time\'] == 1117685909701) ]\nfiltered_ngsim_U.xs((filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == 0 )].index.get_level_values("Relative_Time")[0],filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == 0 )].index.get_level_values("Preceding").values[0]),level=("Relative_Time",\'Vehicle_ID\')).index.get_level_values(\'id\').values[0] \nfiltered_ngsim_U.index.get_level_values("Preceding")\nngsim[(ngsim[\'Vehicle_ID\'] == 242)|(ngsim[\'Preceding\'] == 242) ].to_csv(\'ng\', index = True)\nfiltered_ngsim_U[(filtered_ngsim_U.index.get_level_values("Vehicle_ID") == 242 

In [16]:
filtered_ngsim= filtered_ngsim_U

In [17]:
stringpath= str(p)[0:str(p).rfind('\\')] + '\\data'
ngsimfilteredfile=stringpath + '\\' + 'Sample_Cleaned_NGSIM_Data_Part_2.csv'
filtered_ngsim.to_csv(ngsimfilteredfile, index = True)